# Chapter 4: Separating Data and Instructions

（第4章：データと指示を分離する）

- [レッスン](#lesson)
- [演習](#exercises)
- [例のプレイグラウンド](#example-playground)

## Setup

次のセットアップセルを実行して、APIキーの読み込みと `get_completion` ヘルパー関数の準備を行います。

In [ ]:
%pip install anthropic --quiet

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Import python's built-in regular expression library
import re
from anthropic import AnthropicBedrock

%store -r MODEL_NAME
%store -r AWS_REGION

client = AnthropicBedrock(aws_region=AWS_REGION)

def get_completion(prompt, system=''):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": prompt}
        ],
        system=system
    )
    return message.content[0].text

---

## Lesson

多くの場合、毎回フルのプロンプトを書くのではなく、**あとから追加の入力データを差し込めるプロンプトテンプレート** を用意したいことがあります。たとえば、Claude に毎回同じ作業をさせたい一方で、使うデータはケースごとに異なる、といった場合です。

これは、**固定の骨格（テンプレート）と可変のユーザー入力を分け**、送信前にユーザー入力をテンプレートへ **差し込む（置換する）** ことで簡単に実現できます。

以下では、置換可能なプロンプトテンプレートの作り方と、ユーザー入力を差し込む方法を段階的に説明します。

### Examples

最初の例では、Claude に『動物の鳴き声ジェネレータ』として振る舞ってもらいます。Claude に渡す最終的なプロンプトは、`PROMPT_TEMPLATE` に入力（この例では "Cow"）を差し込んだものです。`ANIMAL` プレースホルダーが、f-string によって "Cow" に置き換わっていることが、フルプロンプトの表示で確認できます。

**Note:** 実際にはプレースホルダー変数名は何でも構いません。この例では `ANIMAL` としましたが、`CREATURE` や `A` でも動きます（ただし、テンプレートが置換前でも読みやすいように、具体的で関連のある変数名にするのがおすすめです）。重要なのは、テンプレートの f-string で使う変数名と一致していることです。

In [ ]:
# Variable content
ANIMAL = "Cow"

# Prompt template with a placeholder for the variable content
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

なぜ入力をこうして分離・差し込みするのでしょうか？ それは、**プロンプトテンプレートが反復作業を簡単にする** からです。たとえば第三者のユーザーに入力してもらう仕組み（この例では『鳴き声を知りたい動物名』）を作る場合、ユーザーはフルのプロンプトを書く必要も、見る必要もありません。変数に値を入れるだけで済みます。

ここでは変数と f-string で差し込みをしていますが、`format()` メソッドでも同じことができます。

**Note:** プロンプトテンプレートには、必要なだけ変数を持たせられます！

このように置換変数を導入する際は、**変数の開始と終了（＝入力データの範囲）** を Claude が確実に理解できるようにすることが非常に重要です（指示やタスク説明と混ざらないようにする）。まずは、指示と置換変数の間に区切りがない例を見てみましょう。

人間の目には、以下のテンプレートで変数がどこからどこまでか明確です。しかし、置換後の完全なプロンプトでは、その境界が曖昧になります。

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. {EMAIL} <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

この場合、**Claude は "Yo Claude" を『書き換えるべきメール本文の一部』だと誤解** してしまいます。出力が "Dear Claude" で始まることから分かります。テンプレート上ではメールの範囲が明確でも、置換後には曖昧になりやすいのです。

どう解決するでしょう？ **入力を XML タグで包む** のが有効です！ 下の例ではそうしており、出力から "Dear Claude" が消えています。

[XML tags](https://docs.anthropic.com/claude/docs/use-xml-tags) は、`<tag></tag>` のような山括弧のタグです。開始タグ（例：`<tag>`）と、`/` の付いた終了タグ（例：`</tag>`）のペアで構成され、`<tag>content</tag>` のように内容を包みます。

**Note:** Claude は多様な区切り文字やデリミタを扱えますが、Claude はプロンプト整理の仕組みとして XML タグを認識するよう特に学習されているため、**区切りとしては XML タグの利用を推奨** します。関数呼び出し以外では、性能を最大化するために使うべき『特別な XML タグ』があるわけではありません。Claude は意図的に、柔軟でカスタマイズしやすいように設計されています。

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. <email>{EMAIL}</email> <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

XML タグが役立つ別の例を見てみましょう。

次のプロンプトでは、**Claude が『指示』と『入力』のどこからどこまでかを誤解** します。フォーマットの影響で、`Each is about an animal, like rabbits` がリストの一部だと誤って解釈されてしまいます（`SENTENCES` 変数を埋めるユーザーは、そこまでを入力に含めたくないはずです）。

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

これを直すには、**ユーザー入力の文（SENTENCES）を XML タグで囲む** だけで十分です。`- Each is about an animal...` の前にあるハイフンが紛らわしくても、入力データの範囲が明確になります。

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f""" Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

**Note:** "Each is about an animal" の誤り例では、この例に必要な誤解を起こさせるために、あえてハイフンを入れています。これはプロンプト作成の重要な教訓です。**小さな細部が結果に影響します**。プロンプト中の誤字・脱字や文法ミスは、常に見直す価値があります。Claude はパターンに敏感で（初期の段階では生のテキスト予測モデルでした）、あなたがミスをするとミスしやすく、賢く書けば賢く、ふざけた書き方をすればふざけた方向に寄る、という傾向があります。

上の内容を変えずにレッスンのプロンプトだけ試したい場合は、ノートブック末尾の [**Example Playground**](#example-playground) を利用してください。

---

## Exercises
- [Exercise 4.1 - Haiku Topic](#exercise-41---haiku-topic)
- [Exercise 4.2 - Dog Question with Typos](#exercise-42---dog-question-with-typos)
- [Exercise 4.3 - Dog Question Part 2](#exercise-42---dog-question-part-2)

### Exercise 4.1 - Haiku Topic
`PROMPT` を、`TOPIC` という変数を受け取り、そのトピックについての俳句を出力するテンプレートに変更してください。この演習は、f-string による変数テンプレートの構造を理解しているか確認するためのものです。

In [ ]:
# Variable content
TOPIC = "Pigs"

# Prompt template with a placeholder for the variable content
PROMPT = f""

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("pigs", text.lower()) and re.search("haiku", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが欲しい場合は、次のセルを実行してください！

In [ ]:
print(hints.exercise_4_1_hint)

### Exercise 4.2 - Dog Question with Typos
XML タグを追加して `PROMPT` を修正し、Claude が正しい答えを出すようにしてください。

それ以外はできるだけ変更しないでください。ぐちゃぐちゃでミスだらけの文章は意図的で、Claude がこうしたミスにどう反応するかを見るためのものです。

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが欲しい場合は、次のセルを実行してください！

In [ ]:
print(hints.exercise_4_2_hint)

### Exercise 4.3 - Dog Question Part 2
XML タグを追加せずに `PROMPT` を修正してください。代わりに、プロンプトから 1〜2語だけ削除します。

上の演習と同様に、ほかはできるだけ変更しないでください。これにより、Claude がどの程度の崩れた言語でも解析・理解できるかが分かります。

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが欲しい場合は、次のセルを実行してください！

In [ ]:
print(hints.exercise_4_3_hint)

### Congrats!

ここまでの演習をすべて解けたら、次の章に進む準備ができています。Happy prompting!

---

## Example Playground

このエリアでは、このレッスンで示したプロンプト例を自由に試し、プロンプトを調整して Claude の応答がどう変わるかを観察できます。

In [ ]:
# Variable content
ANIMAL = "Cow"

# Prompt template with a placeholder for the variable content
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. {EMAIL} <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. <email>{EMAIL}</email> <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f""" Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))